In [1]:
import glob
import numpy as np
import time
import math
import random
from scipy import linalg as LA
import pandas as pd
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.svm import NuSVC
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
import csv

In [2]:
def read_file(file_name):
    data = []
    c = 0
    with open(file_name, "r") as f:
        for line in f:
            c = c + 1
            if c > 6:
                item = line.strip().split(",")
                data.append(np.asarray(list(map(float,item[:2]))))
    return data

In [3]:
def read_file(file_name):
    data = []
    c = 0
    with open(file_name, "r") as f:
        for line in f:
            c = c + 1
            if c > 6:
                item = line.strip().split(",")
                data.append([int(file_name[45:-4])] + list(map(float, item[:2])))
    return data

In [4]:
J = glob.glob('Geolife Trajectories 1.3/**/', recursive=True)[2:]
K = [J[2*i+1] for i in range(182)]
F = [K[i][30:33] for i in range(182)]
int1 = np.vectorize(int)
folder_numbers = int1(F)
I = glob.glob('Geolife Trajectories 1.3/**/*.plt', recursive=True)

In [5]:
# runtime about 65s
Start_time = time.time()
M = 182
data_indexed = [0] * M
data = [0] * M

for i in range(M):
    data_indexed[folder_numbers[i]] = []
    data[folder_numbers[i]] = []
    j = 0
    for file_name in I:
        if file_name[30:33] == F[i]:
            a = np.array(read_file(file_name))
            data[folder_numbers[i]].append(a)
            data_indexed[folder_numbers[i]].append(([j, folder_numbers[i], a]))
            j = j+1
    
print('total time =', time.time() - Start_time)

total time = 93.40509104728699


In [6]:
def read_file_index(file_name):
    a = int(file_name[45:-4])
    return a

In [7]:
read_file_index(I[0])

20090103012134

In [8]:
# Extra
M = 182
traj_indices = [0] * M

for i in range(M):
    traj_indices[folder_numbers[i]] = []

    for file_name in I:
        if file_name[30:33] == F[i]:
            a = read_file_index(file_name)
            traj_indices[folder_numbers[i]].append(([folder_numbers[i], a]))

In [9]:
# runtime about 65s
Start_time = time.time()
M = 182
data_indexed = [0] * M
data = [0] * M

for i in range(M):
    
    data_indexed[folder_numbers[i]] = []
    data[folder_numbers[i]] = []

    for file_name in I:
        if file_name[30:33] == F[i]:
            a = np.array(read_file(file_name))
            data[folder_numbers[i]].append(a)
            data_indexed[folder_numbers[i]].append(([folder_numbers[i], int(a[0][0]), a[:,1:]]))
    
print('total time =', time.time() - Start_time)

total time = 90.15114521980286


In [49]:
data_reduced = [0] * M
data_fol_num = set()

for i in range(M):
    data_reduced[i] = []
    for j in range(len(data_indexed[i])):
        if len(data_indexed[i][j][2]) > 50 and len(data_indexed[i][j][2]) < 200: 
            data_fol_num.add(data_indexed[i][j][1])
            data_reduced[i].append(data_indexed[i][j])

data_fol_num = np.sort(list(data_fol_num))

In [50]:
sum = 0
for i in range(M):
    sum = sum + len(data_reduced[i])
sum

3165

In [53]:
c = 0
A = []
for i in range(M):
    if len(data_reduced[i]) == 0:
        c = c + 1
        A.append(i)
c, A

(34,
 [27,
  31,
  33,
  35,
  48,
  49,
  50,
  54,
  59,
  60,
  74,
  77,
  87,
  100,
  106,
  109,
  113,
  116,
  118,
  120,
  123,
  130,
  137,
  143,
  146,
  148,
  149,
  151,
  156,
  160,
  165,
  169,
  170,
  177])

In [ ]:
l = 0
index = [] 
for i in range(182):
    if len(data_reduced[i]) < 10: #10
        l = l + 1
    else:
        index.append(i)
        
l, 182-l

In [18]:
k = 20
Q_normal = np.ones((k,2))

mu_x = 39.6
sigma_x = 1.4
Q_normal[:,0] = np.random.normal(mu_x, sigma_x, k)

mu_y = 114.8
sigma_y = 9.4
Q_normal[:,1] = np.random.normal(mu_y, sigma_y, k)

In [14]:
# We have used this Q in the paper

Q = np.array([[39.6,114.8],[40,116.34], [39, 114], [40,115], [41,116], [39.5,113.5], 
                [40.5, 115.5], [40.3, 114.4], [38.5, 113.5], [39.3,117], [39.5,115], 
                [39.4, 116], [39,114.6], [40.5, 115.5], [41.2,115], [42, 116], [41.7, 114.8], 
               [38,115], [37, 116], [37.5, 114]])

#np.savetxt('Q.csv', Q, delimiter=',')
#np.savetxt('Q.txt', Q, delimiter=',')

In [10]:
Q = np.array([[39.98238166771571, 116.59378051757814],
            [40.09067983779908, 116.41937255859376],
            [40.02025300564684, 116.15844726562501],
            [39.829631721333755, 116.18179321289064],
            [39.76632525654491, 116.36993408203126],
            [39.828577091142016, 116.61163330078126],
            [39.88128876522249, 116.50039672851564],
            [39.9665957444875, 116.48391723632814],
            [39.987642831840844, 116.42898559570312],
            [39.99711190624385, 116.34384155273439],
            [39.96449067924025, 116.29028320312501],
            [39.89814795018363, 116.28890991210939],
            [39.88234258575577, 116.40563964843751],
            [39.939224840791965, 116.44409179687501],
            [39.96554321996606, 116.37268066406251],
            [39.9476478239225, 116.33834838867189],
            [39.929747745342944, 116.38229370117189],
            [40.351413291536915, 116.5312045812607],
            [40.213481238893465, 116.06591105461122],
            [39.424976686338404, 116.68437480926515]])

In [14]:
def fun(x):
    if x >= 1: 
        y = 0
    elif x <= -1:
        y = math.pi
    else: 
        y = np.arccos(x)
    return y

In [15]:
def dist_signed_point_closed(Q, gamma, sigma): 
    
    p1 = gamma[:-1]
    p2 = gamma[1:]
    L = np.sqrt(((p2-p1)*(p2-p1)).sum(axis =1)) + 10e-6
    
    w = (p1-p2)*(-1,1)/(L * np.ones((2,1))).T
    w[:,[0, 1]] = w[:,[1, 0]]
    
# signed distance to the extended lines of segments
    dist_signed = np.sum(w * (Q.reshape(len(Q),1,2) - p1), axis=2)
    x = abs(dist_signed.copy())
    R = (L**2).reshape(-1,1)
# u = argmin points on the extended lines of segments
    u = p1 + ((((np.sum(((Q.reshape(len(Q),1,2) - p1) * (p2 - p1)),axis=2).reshape(len(Q)
                ,-1,1,1) * (p2-p1).reshape(len(p2-p1),1,2))).reshape(len(Q),len(p1),2))/R)

    G = np.sqrt(np.sum((u-p1)*(u-p1), axis=2))
    H = np.sqrt(np.sum((u-p2)*(u-p2), axis=2))
# d1 = distance to start points
    d1 = np.sqrt(np.sum((Q.reshape(len(Q),1,2)-p1)*(Q.reshape(len(Q),1,2)-p1), axis=2))
# d2 = distance to end points
    d2 = np.sqrt(np.sum((Q.reshape(len(Q),1,2)-p2)*(Q.reshape(len(Q),1,2)-p2), axis=2))
    d = np.where(d1 < d2, d1, d2)
    dist_segment = np.where(abs(G + H - L) < np.ones(len(L)) * (10e-6), dist_signed, d)
    
    J2 = [0] * len(Q)
    for i in range(len(Q)): 
        J2[i] = np.where(abs(G + H - L)[i] > 10e-6)[0]
    J2 = np.array(J2)

    dist_segment_copy = dist_segment.copy()
    dist = abs(dist_segment_copy)


    j = np.argmin(dist, axis =1)

    sign = np.ones(len(Q))
    for k in range(len(Q)): 
        if j[k] in J2[k]:
            if j[k] == 0 and LA.norm(Q[k] - gamma[0]) < LA.norm(Q[k] - gamma[1]):
                
                y = LA.norm(gamma[0]-gamma[1]) - LA.norm(gamma[-1] - gamma[-2])
                if y < 0:
                    x = gamma[0] + 0.1 * LA.norm(gamma[0]-gamma[1])*(gamma[-2]-gamma[-1])/LA.norm(gamma[-2]-gamma[-1])
                    z = gamma[0] + 0.1 * LA.norm(gamma[0]-gamma[1])*(gamma[1]-gamma[0])/LA.norm(gamma[1]-gamma[0])
                    q = 2 * gamma[0] - (x + z)/2
                else: 
                    x = gamma[0] + 0.1 * LA.norm(gamma[-1]-gamma[-2])*(gamma[1]-gamma[0])
                    z = gamma[0] + 0.1 * LA.norm(gamma[-1]-gamma[-2])*(gamma[-2]-gamma[-1])
                    q = 2 * gamma[0] - (x + z)/2
                sign[k] = np.sign((q-gamma[-1]).dot(w[-1] + w[0]))
                
            elif j[k] == len(gamma)-2 and LA.norm(Q[k] - gamma[-1]) < LA.norm(Q[k] - gamma[-2]):
                s = w[-1].dot((Q[k] - gamma[-1])/ LA.norm(Q[k] - gamma[-1]) + 10e-6)
                sign[k] = np.sign(s)
            
            elif LA.norm(Q[k] - gamma[j[k]]) < LA.norm(Q[k] - gamma[j[k]+1]):  
                q = 2 * gamma[j[k]] - (gamma[j[k]-1] + gamma[j[k]+1])/2
                sign[k] = np.sign((q-gamma[j[k]]).dot(w[j[k]-1] + w[j[k]]))
                    
            elif LA.norm(Q[k] - gamma[j[k]+1]) <= LA.norm(Q[k] - gamma[j[k]]):
                q = 2 * gamma[j[k]+1] - (gamma[j[k]] + gamma[j[k]+2])/2
                sign[k] = np.sign((q-gamma[j[k]+1]).dot(w[j[k]] + w[j[k]+1]))

    E = dist_segment[np.arange(len(dist_segment)),j] 
    F = dist[np.arange(len(dist)),j] 
    dist_weighted = sign * (1/sigma) * (E.reshape(-1,1) * np.exp(-(F/sigma)**2).reshape(-1,1)).reshape(1,-1)

    return dist_weighted.reshape(len(Q))

In [16]:
def dist_signed_point_unclosed(Q, gamma, sigma): 
    
    p1 = gamma[:-1]
    p2 = gamma[1:]
    #L = np.sqrt(((p2-p1)*(p2-p1)).sum(axis =1))
    #II = np.where(L > 10e-6)[0]
    #L = L[II]
    #p1 = p1[II]
    #p2 = p2[II]
    #gamma = np.array(list(p1) + [p2[-1]])
    L = np.sqrt(((p2-p1)*(p2-p1)).sum(axis =1)) + 10e-6
    w = (p1-p2)*(-1,1)/(L * np.ones((2,1))).T
    w[:,[0, 1]] = w[:,[1, 0]]
    
# signed distance to the extended lines of segments
    dist_signed = np.sum(w * (Q.reshape(len(Q),1,2) - p1), axis=2)
    x = abs(dist_signed.copy())
    R = (L**2).reshape(-1,1)
# u = argmin points on the extended lines of segments
    u = p1 + ((((np.sum(((Q.reshape(len(Q),1,2) - p1) * (p2 - p1)),axis=2).reshape(len(Q)
                ,-1,1,1) * (p2-p1).reshape(len(p2-p1),1,2))).reshape(len(Q),len(p1),2))/R)

    G = np.sqrt(np.sum((u-p1)*(u-p1), axis=2))
    H = np.sqrt(np.sum((u-p2)*(u-p2), axis=2))
# d1 = distance to start points
    d1 = np.sqrt(np.sum((Q.reshape(len(Q),1,2)-p1)*(Q.reshape(len(Q),1,2)-p1), axis=2))
# d2 = distance to end points
    d2 = np.sqrt(np.sum((Q.reshape(len(Q),1,2)-p2)*(Q.reshape(len(Q),1,2)-p2), axis=2))
    d = np.where(d1 < d2, d1, d2)
    dist_segment = np.where(abs(G + H - L) < np.ones(len(L)) * (10e-6), dist_signed, d)
    
    J2 = [0] * len(Q)
    for i in range(len(Q)): 
        J2[i] = np.where(abs(G + H - L)[i] > 10e-6)[0]
    J2 = np.array(J2)

    dist_segment_copy = dist_segment.copy()
    dist = abs(dist_segment_copy)
    
    dist_from_start_1 = np.sqrt(((Q -p1[0])*(Q -p1[0])).sum(axis =1))
    ds_1 = ((Q -p1[0])*w[0]).sum(axis =1)
    #dist_from_start = np.sign(ds_1) * (abs(ds_1) + np.sqrt(dist_from_start_1**2 - ds_1**2 + 10e-6))
    dist_from_start = ds_1 * np.maximum(abs(ds_1), np.sqrt(dist_from_start_1**2 - ds_1**2 + 10e-6))/ (dist_from_start_1 + 10e-6)

    
    dist_from_end_1 = np.sqrt(((Q -p2[-1])*(Q -p2[-1])).sum(axis =1))
    de_1 = ((Q -p2[-1])* w[-1]).sum(axis =1)
    #dist_from_end = np.sign(de_1) * (abs(de_1) + np.sqrt(dist_from_end_1**2 - de_1**2 + 10e-6))
    dist_from_end = de_1 * np.maximum(abs(de_1), np.sqrt(dist_from_end_1**2 - de_1**2 + 10e-6))/ (dist_from_end_1+ 10e-6)


    dist_segment[:,0] = np.where(abs(dist[:,0]- dist_from_start_1)< 10e-8, dist_from_start, dist_segment[:,0]) 
    dist_segment[:,-1] = np.where(abs(dist[:,-1]- dist_from_end_1)< 10e-8, dist_from_end, dist_segment[:,-1]) 


    j = np.argmin(dist, axis =1)

    sign = np.ones(len(Q))
    for k in range(len(Q)): 
        if j[k] in J2[k]: 
            if j[k] == 0 and LA.norm(Q[k] - gamma[0]) < LA.norm(Q[k] - gamma[1]):
                sign[k] = 1
                
            elif j[k] == len(gamma)-2 and LA.norm(Q[k] - gamma[j[k]+1]) < LA.norm(Q[k] - gamma[j[k]]):
                sign[k] = 1
            
            elif LA.norm(Q[k] - gamma[j[k]]) < LA.norm(Q[k] - gamma[j[k]+1]):  
                q = 2 * gamma[j[k]] - (gamma[j[k]-1] + gamma[j[k]+1])/2
                sign[k] = np.sign((q-gamma[j[k]]).dot(w[j[k]-1] + w[j[k]]))
                    
            elif LA.norm(Q[k] - gamma[j[k]+1]) <= LA.norm(Q[k] - gamma[j[k]]) and j[k]+2 <=len(gamma)-1:
                q = 2 * gamma[j[k]+1] - (gamma[j[k]] + gamma[j[k]+2])/2
                sign[k] = np.sign((q-gamma[j[k]+1]).dot(w[j[k]] + w[j[k]+1]))

    E = dist_segment[np.arange(len(dist_segment)),j] 
    F = dist[np.arange(len(dist)),j] 
    dist_weighted = sign * (1/sigma) * (E.reshape(-1,1) * np.exp(-(F/sigma)**2).reshape(-1,1)).reshape(1,-1)

    return dist_weighted.reshape(len(Q))

In [17]:
def dist_signed_point(Q, gamma, sigma):
    if LA.norm(gamma[0]-gamma[-1]) > 10e-6:
        A = dist_signed_point_unclosed(Q, gamma, sigma)
    else: 
        A = dist_signed_point_closed(Q, gamma, sigma)
        
    return A

In [106]:
Q = np.array([[39.960873530470465, 116.33834838867189], 
                [39.99353485464963, 116.38229370117189],
                [39.9893213674195, 116.28341674804689],
                [39.93979342478733, 116.28067016601562],
                [39.91237957606774, 116.33834838867189],
                [39.925033485662496, 116.40014648437501],
                [39.96298118372819, 116.43035888671875],
                [40.02933900095626, 116.45507812500001],
                [40.04618151168254, 116.39053344726564],
                [40.05670596834186, 116.29577636718751],
                [39.98405414287271, 116.22573852539064],
                [39.89972332832801, 116.21612548828126],
                [39.88073457310286, 116.43859863281251],
                [39.78254222723113, 116.33422851562501],
                [39.83746277453909, 116.25595092773439],
                [40.138740946756734, 116.26144409179689],
                [40.25216451805882, 116.18316650390626],
                [40.105097535318116, 116.14334106445314],
                [40.003014250556554, 116.90277099609376],
                [39.863851263932176, 116.70776367187501]])

In [47]:
Q = np.array([[40.02784036172628, 116.50178611278535],
[40.085652349925475, 116.39329612255098],
[40.08039874140986, 116.24772727489473],
[40.05202225054736, 116.19966208934785],
[39.97418906423696, 116.1694496870041],
[39.899427860534345, 116.16807639598848],
[39.831967504781055, 116.2518471479416],
[39.82880367321281, 116.3617104291916],
[39.87097611205365, 116.4935463666916],
[39.95419054874236, 116.43724143505098],
[40.00575375653265, 116.37956321239473],
[40.024685570075626, 116.31501853466034],
[39.9699793365808, 116.2738198041916],
[39.90996254870152, 116.2793129682541],
[39.91733586660022, 116.3617104291916],
[39.967874375523536, 116.34385764598848],
[39.71269591644774, 116.47981345653535],
[39.66091215703148, 116.37407004833223],
[39.6989611532183, 116.2573403120041],
[39.74226919672803, 116.06095969676973],
[39.90996254870152, 116.01426780223848],
[40.129766654747705, 116.07743918895723],
[40.21895804658573, 116.0760658979416],
[40.257745909206854, 116.25596702098848],
[40.31640959147105, 116.10215842723846],
[40.20008024183078, 116.58418357372285],
[40.07724638171063, 116.79155051708223],
[39.897320728525514, 116.76133811473848],
[40.095087305350994, 116.98068380355835],
[39.93416561718383, 117.04934835433961],
[39.60272060950412, 117.02874898910524],
[39.537087506313945, 116.85022115707399],
[39.32177827012156, 116.49766623973848],
[39.35045657030925, 116.33699119091035],
[39.45338894346977, 116.05271995067598],
[40.030995007481806, 116.04585349559785],
[40.28917921376592, 116.4660805463791],
[40.41685741933303, 115.90852439403535],
[39.41933055574406, 116.00316211581233],
[39.4341814883212, 115.56096240878108],
[38.994696721865594, 115.50629228353503],
[39.1578093485124, 115.17120927572252],
[38.91460188981307, 115.06134599447253],
[39.96682187068797, 115.84260642528535],
[40.444612929944775, 116.78323835134509],
[39.037848825595745, 117.04134196043016],
[39.18809007444354, 117.30501383543016],
[39.65284221780906, 116.5751498937607],
[39.80176509832639, 115.9763950109482], 
[39.70176509836639, 116.9763900109482]])

In [48]:
r = 182
sigma = 0.2
projected_curves = [0] * r # in R^20

In [49]:
Start_time = time.time()

for i in range(M):
    Start_time_1 = time.time()
    projected_curves[i] = []
    for j in range(len(data_indexed[i])):
        projected_curves[i].append([data_indexed[i][j][0], data_indexed[i][j][1]]+ 
                                    list(dist_signed_point(Q,data_indexed[i][j][2],sigma)))
    print(i,'time =', time.time() - Start_time_1)

print('total time =', time.time() - Start_time)

0 time = 3.4587457180023193
1 time = 2.2791452407836914
2 time = 5.336399078369141
3 time = 9.968201875686646
4 time = 8.845421075820923
5 time = 2.364084005355835
6 time = 0.6056680679321289
7 time = 1.744271993637085
8 time = 1.5681848526000977
9 time = 1.5716018676757812
10 time = 20.898401021957397
11 time = 1.8856220245361328
12 time = 3.1637799739837646
13 time = 5.414434909820557
14 time = 7.536698818206787
15 time = 1.8305070400238037
16 time = 1.7443897724151611
17 time = 21.099974155426025
18 time = 0.9598438739776611
19 time = 1.0177159309387207
20 time = 3.491316080093384
21 time = 0.05370688438415527
22 time = 9.594305992126465
23 time = 2.440412998199463
24 time = 5.115517854690552
25 time = 12.550628900527954
26 time = 3.020345687866211
27 time = 0.30547475814819336
28 time = 2.4187979698181152
29 time = 1.379309892654419
30 time = 12.216701984405518
31 time = 0.4263930320739746
32 time = 0.5037519931793213
33 time = 1.471465826034546
34 time = 3.2488062381744385
35 time

In [50]:
for i in range(M):
    np.savetxt('3/2+50-dim-traj-Nov-27/'+str(i)+'.csv', projected_curves[i], delimiter=',')
    #np.savetxt('Temp_Vectors/V_Qs for trajectories of user '+str(data_fol_num1[i])+'.csv', projected_curves[i], delimiter=',')
    #np.savetxt('Distance_Matrices_Q_sigma=0.3/Distances d_Q between trajectories of user '+str(data_fol_num1[i])+'.csv', D[i], delimiter=',')
        

In [51]:
total_num_traj = len(I)
final_projected_data = np.zeros((total_num_traj, 2 + len(Q)))

k = 0
for i in range(M):
    for j in range(len(projected_curves[i])):
        final_projected_data[k] = projected_curves[i][j]
        k = k +1

In [52]:
final_projected_data.shape

(18670, 52)

In [73]:
np.savetxt('3/2+50-dim-all-in-1-file-Nov-27.csv', 
           final_projected_data, delimiter=',')

In [74]:
path2 = '3/2+50-dim-all-in-1-file-Nov-27.csv'
path3 = '3/2+50-dim-all-in-1-file-Nov-27_1.csv'

head = ['User_Name', 'Trajectory', 'x_1', 'x_2', 'x_3', 'x_4', 'x_5', 'x_6', 'x_7', 'x_8', 
       'x_9', 'x_10', 'x_11', 'x_12', 'x_13', 'x_14', 'x_15', 'x_16', 'x_17', 'x_18',
       'x_19', 'x_20', 'x_21', 'x_22', 'x_23', 'x_24', 'x_25', 'x_26', 'x_27', 'x_28', 
       'x_29', 'x_30', 'x_31', 'x_32', 'x_33', 'x_34', 'x_35', 'x_36', 'x_37', 'x_38',
       'x_39', 'x_40','x_41', 'x_42', 'x_43', 'x_44', 'x_45', 'x_46', 'x_47', 'x_48', 
       'x_49', 'x_50']

with open(path2, "w", newline='') as f:
    writer = csv.writer(f, delimiter=',')
    writer.writerow(head) # write the header

    # write the actual content line by line
    with open(path3, 'r', newline='') as incsv:
        reader = csv.reader(incsv)
        writer.writerows(row for row in reader) 

# Dimensionality Reduction with PCA

In [75]:
X = final_projected_data[:,2:]
pca = PCA(n_components=2)
pca.fit(X)
X_Reduced = pca.transform(X)
np.savetxt('3/2-dim-all-in-1-file-Nov-27.csv', X_Reduced, delimiter=',')

#print(pca.explained_variance_ratio_)
#print(pca.singular_values_)

In [76]:
print(pca.singular_values_)

[85.89898949 75.08087803]


In [77]:
W = np.concatenate((final_projected_data[:,:2], X_Reduced), axis =1)
np.savetxt('3/50+2-to-2+2-PCA.csv', W, delimiter=',')


In [78]:
path4 = '3/50+2-to-2+2-PCA.csv'
path5 = '3/50+2-to-2+2-PCA_1.csv'

head = ['User_Name', 'Trajectory', 'x', 'y']

with open(path4, "w", newline='') as f:
    writer = csv.writer(f, delimiter=',')
    writer.writerow(head) # write the header

    # write the actual content line by line
    with open(path5, 'r', newline='') as incsv:
        reader = csv.reader(incsv)
        writer.writerows(row for row in reader) 

# Old distance (without direction)

In [150]:
def old_dist(Q, gamma):
    
    p2 = gamma[1:]
    p1 = gamma[:-1]
    L = np.sqrt(((p2-p1)*(p2-p1)).sum(axis =1)) + 10e-6
    #II = np.where(L>10e-8)[0]
    #L = L[II]
    #p1 = p1[II]
    #p2 = p2[II]
    w = (p1-p2)*(-1,1)/(L*np.ones((2,1))).T
    w[:,[0, 1]] = w[:,[1, 0]]
    
    dist_dot = np.sum(w * (Q.reshape(len(Q),1,2) - p1), axis=2)
    
    x = abs(dist_dot.copy())
    R = (L**2).reshape(-1,1)
    u = p1 + ((((np.sum(((Q.reshape(len(Q),1,2) - p1) * (p2 - p1)),axis=2).reshape(len(Q)
                ,-1,1,1) * (p2-p1).reshape(len(p2-p1),1,2))).reshape(len(Q),len(p1),2))/R)
    
    G = np.sqrt(np.sum((u-p1)*(u-p1), axis=2))
    H = np.sqrt(np.sum((u-p2)*(u-p2), axis=2))
    d1 = np.sqrt(np.sum((Q.reshape(len(Q),1,2)-p1)*(Q.reshape(len(Q),1,2)-p1), axis=2))
    d2 = np.sqrt(np.sum((Q.reshape(len(Q),1,2)-p2)*(Q.reshape(len(Q),1,2)-p2), axis=2))

    dist = np.where(abs(G + H - L) < np.ones(len(L)) * (10e-8), x, np.minimum(d1, d2))

    j = np.argmin(dist, axis =1)
    dist_weighted = dist[np.arange(len(dist)),j] # np.choose(j, dist.T) 
    
    return dist_weighted.reshape(len(Q)) # (w[j],dist[j],dist_dot[j],dist_weighted)

In [152]:
old_proj_curves = [0] * M # in R^20

In [153]:
Start_time = time.time()
for i in range(M):
    Start_time_1 = time.time()
    old_proj_curves[i] = []
    for j in range(len(data_final[i])):
        old_proj_curves[i].append([data_indexed[i][j][0], data_indexed[i][j][1]]+ 
                                    list(old_dist(Q,data_final[i][j][2])))
    print(i,'time =', time.time() - Start_time_1)

print('total time =', time.time() - Start_time)

0 time = 0.8398172855377197
1 time = 0.5506978034973145
2 time = 1.218080997467041
3 time = 2.369197130203247
4 time = 2.192682981491089
5 time = 0.5791020393371582
6 time = 0.15708088874816895
7 time = 0.4302840232849121
8 time = 0.36307811737060547
9 time = 0.38161516189575195
10 time = 5.321523904800415
11 time = 0.4254157543182373
12 time = 0.7454369068145752
13 time = 1.3728671073913574
14 time = 1.8480980396270752
15 time = 0.4246830940246582
16 time = 0.42302989959716797
17 time = 5.16010308265686
18 time = 0.28659582138061523
19 time = 0.2625572681427002
20 time = 0.9476132392883301
21 time = 0.012360334396362305
22 time = 2.34653902053833
23 time = 0.6321001052856445
24 time = 1.201584815979004
25 time = 2.8755998611450195
26 time = 0.7881529331207275
27 time = 0.0744330883026123
28 time = 0.6233248710632324
29 time = 0.33955883979797363
30 time = 2.9802842140197754
31 time = 0.10332512855529785
32 time = 0.1310129165649414
33 time = 0.37662506103515625
34 time = 0.81177091598

In [154]:
for i in range(r):
    np.savetxt('Vis_Vectors_old-2+20-dimensional-Nov-26/'+str(data_fol_num1[i])+'.csv', 
               old_proj_curves[i], delimiter=',')
    #np.savetxt('Old_Vectors_2/V_Qs for trajectories of user '+str(data_fol_num1[i])+'.csv', 
     #          old_proj_curves[i], delimiter=',')

In [156]:
total_num_traj = len(I)
final_proj_old_data = np.zeros((total_num_traj, 2 + len(Q)))

k = 0
for i in range(M-l):
    for j in range(len(old_proj_curves[i])):
        final_proj_old_data[k] = old_proj_curves[i][j]
        k = k +1

In [159]:
final_proj_old_data.shape

(18670, 22)

# Dimensionality Reduction with PCA

In [160]:
Y = final_proj_old_data[:,2:]
pca = PCA(n_components=2)
pca.fit(Y)
Y_Reduced = pca.transform(Y)
np.savetxt('Vis_Vectors_old-2+20-to-2+2-PCA-Nov-26/Vis_Vectors_old-2+20-to-2+2-PCA-Nov-26.csv', Y_Reduced, delimiter=',')


In [161]:
print(pca.singular_values_)

[12869.75919026    31.14062816]


In [171]:
Z = np.concatenate((final_proj_old_data[:,:2], Y_Reduced), axis =1)
np.savetxt('Vis_Vectors_old-2+20-to-2+2-PCA-Nov-26/Vis_Vectors_old-2+20-to-2+2-PCA-indexed-Nov-26.csv', 
           Z, delimiter=',')
